DB에 3월 7일 이전 데이터가 존재하지 않아, 이전 데이터를 처리 후 삽입

영업소별 교통량 데이터 다운로드: https://data.ex.co.kr/portal/fdwn/view?type=TCS&num=34&requestfrom=dataset

In [302]:
import pandas as pd
import sqlalchemy as sa
from dotenv import dotenv_values

env = dotenv_values('.env')

## Load

In [303]:
df = pd.read_csv('../data/traffic_data/before/202502_month.csv')
df

,집계일자,집계시,영업소코드,입출구구분코드,TCS하이패스구분코드,고속도로운영기관구분코드,영업형태구분코드,1종교통량,2종교통량,3종교통량,4종교통량,5종교통량,6종교통량,총교통량,Unnamed: 14
0,2025-02-06,15,544,1,2,0,0,245,8,9,12,47,7,328,NaN
1,2025-02-06,15,545,0,1,0,0,12,0,0,0,0,0,12,NaN
2,2025-02-06,15,545,0,2,0,0,127,4,8,3,3,4,149,NaN
3,2025-02-06,15,545,1,1,0,0,16,0,2,1,0,1,20,NaN
4,2025-02-06,15,545,1,2,0,0,151,4,9,8,3,1,176,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119118,2025-02-25,10,985,1,1,0,0,18,1,4,3,1,4,31,NaN
1119119,2025-02-25,10,985,1,2,0,0,208,10,3,7,11,8,247,NaN
1119120,2025-02-25,10,986,0,1,0,0,14,1,4,0,4,0,23,NaN
1119121,2025-02-25,10,986,0,2,0,0,160,2,6,9,28,3,208,NaN


In [304]:
POSTGRES_USER = env.get('POSTGRES_USER')
POSTGRES_PASSWORD = env.get('POSTGRES_PASSWORD')
POSTGRES_HOST = env.get('POSTGRES_HOST')
POSTGRES_PORT = env.get('POSTGRES_PORT')
POSTGRES_DB = env.get('POSTGRES_DB')

engine = sa.create_engine(f'postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}')

In [305]:
with engine.connect() as conn:
    query = '''
        SELECT 
            u.unit_id, 
            u.unit_name,
            tdbu.center_code,
            tdbu.center_name,
            tdbu.brof_code,
            tdbu.brof_name
        FROM 
            unit u
        JOIN (
            SELECT
                tdbu.unit_code,
                tdbu.center_code,
                tdbu.center_name,
                tdbu.brof_code,
                tdbu.brof_name
            FROM
                traffic_data_by_unit tdbu 
            GROUP BY
                tdbu.unit_code,
                tdbu.center_code,
                tdbu.center_name,
                tdbu.brof_code,
                tdbu.brof_name
        ) tdbu
        ON
            u.unit_id = tdbu.unit_code
        WHERE 
            u.unit_id != '0'
        ORDER BY 
            u.unit_id
    '''
    result = conn.execute(query).fetchall()
    unit_df = pd.DataFrame(result)

unit_df.head()

,unit_id,unit_name,center_code,center_name,brof_code,brof_name
0,101,서울,901,서울경기본부,210300,수원
1,102,동수원,901,서울경기본부,210200,군포
2,103,수원신갈,901,서울경기본부,210300,수원
3,104,지곡,906,부산경남본부,710600,산청
4,105,기흥,901,서울경기본부,210300,수원


## Preprocessing

In [306]:
df.drop(df.columns[-1], inplace=True, axis=1)
df

,집계일자,집계시,영업소코드,입출구구분코드,TCS하이패스구분코드,고속도로운영기관구분코드,영업형태구분코드,1종교통량,2종교통량,3종교통량,4종교통량,5종교통량,6종교통량,총교통량
0,2025-02-06,15,544,1,2,0,0,245,8,9,12,47,7,328
1,2025-02-06,15,545,0,1,0,0,12,0,0,0,0,0,12
2,2025-02-06,15,545,0,2,0,0,127,4,8,3,3,4,149
3,2025-02-06,15,545,1,1,0,0,16,0,2,1,0,1,20
4,2025-02-06,15,545,1,2,0,0,151,4,9,8,3,1,176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119118,2025-02-25,10,985,1,1,0,0,18,1,4,3,1,4,31
1119119,2025-02-25,10,985,1,2,0,0,208,10,3,7,11,8,247
1119120,2025-02-25,10,986,0,1,0,0,14,1,4,0,4,0,23
1119121,2025-02-25,10,986,0,2,0,0,160,2,6,9,28,3,208


In [307]:
columns = [
    'std_date',
    'std_hour',
    'unit_code',
    'inout_type',
    'tcs_hipass_type',
    'hiway_type',
    'operator_type',
    '1_traffic_amount',
    '2_traffic_amount',
    '3_traffic_amount',
    '4_traffic_amount',
    '5_traffic_amount',
    '6_traffic_amount',
    'total_traffic_amount',
]

df.columns = columns
df.head()

,std_date,std_hour,unit_code,inout_type,tcs_hipass_type,hiway_type,operator_type,1_traffic_amount,2_traffic_amount,3_traffic_amount,4_traffic_amount,5_traffic_amount,6_traffic_amount,total_traffic_amount
0,2025-02-06,15,544,1,2,0,0,245,8,9,12,47,7,328
1,2025-02-06,15,545,0,1,0,0,12,0,0,0,0,0,12
2,2025-02-06,15,545,0,2,0,0,127,4,8,3,3,4,149
3,2025-02-06,15,545,1,1,0,0,16,0,2,1,0,1,20
4,2025-02-06,15,545,1,2,0,0,151,4,9,8,3,1,176


In [308]:
# Remove front and back white spaces from unit_code column
df['unit_code'] = df['unit_code'].fillna('').astype(str)
df['unit_code'] = df['unit_code'].str.strip()

# inout_type = 0: 입구, 1: 출구
df['inout_name'] = df['inout_type'].map({0: '입구', 1: '출구'})
df.drop(columns=['inout_type'], inplace=True)

# Combine std_date and std_hour
df['std_hour'] = df['std_hour'].astype(str)
df['std_date_str'] = df['std_date'] + ' ' + df['std_hour'] + ':00:00'
df['std_date'] = pd.to_datetime(df['std_date_str'], format='%Y-%m-%d %H:%M:%S')
df.drop(columns=['std_date_str','std_hour'], inplace=True)

df.head()

,std_date,unit_code,tcs_hipass_type,hiway_type,operator_type,1_traffic_amount,2_traffic_amount,3_traffic_amount,4_traffic_amount,5_traffic_amount,6_traffic_amount,total_traffic_amount,inout_name
0,2025-02-06 15:00:00,544,2,0,0,245,8,9,12,47,7,328,출구
1,2025-02-06 15:00:00,545,1,0,0,12,0,0,0,0,0,12,입구
2,2025-02-06 15:00:00,545,2,0,0,127,4,8,3,3,4,149,입구
3,2025-02-06 15:00:00,545,1,0,0,16,0,2,1,0,1,20,출구
4,2025-02-06 15:00:00,545,2,0,0,151,4,9,8,3,1,176,출구


In [ ]:
# 각 차종(1종~6종)으로 나누고 새로운 열을 추가하여 값을 매핑
melted_df = pd.melt(
    df,
    id_vars=['std_date', 'unit_code', 'tcs_hipass_type', 'hiway_type', 'operator_type', 'inout_name'],
    value_vars=['1_traffic_amount', '2_traffic_amount', '3_traffic_amount', '4_traffic_amount', '5_traffic_amount', '6_traffic_amount'],
    var_name='tcs_car_type_cd',
    value_name='traffic_amount'
)

melted_df['tcs_car_type_cd'] = melted_df['tcs_car_type_cd'].str.extract(r'(\d+)')
car_type_mapping = {
    '1': {'tcs_car_type_grp_nm': '소형차', 'tcs_car_type_nm': '1종', 'tcs_car_type_grp_cd': '1'},
    '2': {'tcs_car_type_grp_nm': '중형차', 'tcs_car_type_nm': '2종', 'tcs_car_type_grp_cd': '2'},
    '3': {'tcs_car_type_grp_nm': '대형차', 'tcs_car_type_nm': '3종', 'tcs_car_type_grp_cd': '3'},
    '4': {'tcs_car_type_grp_nm': '대형차', 'tcs_car_type_nm': '4종', 'tcs_car_type_grp_cd': '3'},
    '5': {'tcs_car_type_grp_nm': '대형차', 'tcs_car_type_nm': '5종', 'tcs_car_type_grp_cd': '3'},
    '6': {'tcs_car_type_grp_nm': '소형차', 'tcs_car_type_nm': '6종', 'tcs_car_type_grp_cd': '1'}
}
melted_df[['tcs_car_type_grp_nm', 'tcs_car_type_nm', 'tcs_car_type_grp_cd']] = melted_df['tcs_car_type_cd'].apply(
    lambda x: pd.Series(car_type_mapping[x])
)
melted_df


,std_date,unit_code,tcs_hipass_type,hiway_type,operator_type,inout_name,tcs_car_type_cd,traffic_amount
0,2025-02-06 15:00:00,544,2,0,0,출구,1,245
1,2025-02-06 15:00:00,545,1,0,0,입구,1,12
2,2025-02-06 15:00:00,545,2,0,0,입구,1,127
3,2025-02-06 15:00:00,545,1,0,0,출구,1,16
4,2025-02-06 15:00:00,545,2,0,0,출구,1,151
...,...,...,...,...,...,...,...,...
6714733,2025-02-25 10:00:00,985,1,0,0,출구,6,4
6714734,2025-02-25 10:00:00,985,2,0,0,출구,6,8
6714735,2025-02-25 10:00:00,986,1,0,0,입구,6,0
6714736,2025-02-25 10:00:00,986,2,0,0,입구,6,3


In [ ]:
# 영업소 df와 merge 하여 traffic_data_by_unit table에 저장할 데이터로 가공
joined_df = pd.merge(left=melted_df, right=unit_df, how='left', left_on='unit_code', right_on='unit_id')
joined_df.head()

,std_date,unit_code,tcs_hipass_type,hiway_type,operator_type,inout_name,tcs_car_type_cd,traffic_amount,tcs_car_type_grp_nm,tcs_car_type_nm,tcs_car_type_grp_cd,unit_id,unit_name,center_code,center_name,brof_code,brof_name
0,2025-02-06 15:00:00,544,2,0,0,출구,1,245,소형차,1종,1,544,완주,903.1,전북본부,810400,논산
1,2025-02-06 15:00:00,545,1,0,0,입구,1,12,소형차,1종,1,545,소양,903.1,전북본부,810700,진안
2,2025-02-06 15:00:00,545,2,0,0,입구,1,127,소형차,1종,1,545,소양,903.1,전북본부,810700,진안
3,2025-02-06 15:00:00,545,1,0,0,출구,1,16,소형차,1종,1,545,소양,903.1,전북본부,810700,진안
4,2025-02-06 15:00:00,545,2,0,0,출구,1,151,소형차,1종,1,545,소양,903.1,전북본부,810700,진안


In [312]:
joined_df.columns

Index(['std_date', 'unit_code', 'tcs_hipass_type', 'hiway_type',
       'operator_type', 'inout_name', 'tcs_car_type_cd', 'traffic_amount',
       'tcs_car_type_grp_nm', 'tcs_car_type_nm', 'tcs_car_type_grp_cd',
       'unit_id', 'unit_name', 'center_code', 'center_name', 'brof_code',
       'brof_name'],
      dtype='object')

In [313]:
joined_df.drop(['tcs_hipass_type', 'hiway_type', 'operator_type', 'unit_id'], axis=1, inplace=True)
joined_df.columns

Index(['std_date', 'unit_code', 'inout_name', 'tcs_car_type_cd',
       'traffic_amount', 'tcs_car_type_grp_nm', 'tcs_car_type_nm',
       'tcs_car_type_grp_cd', 'unit_name', 'center_code', 'center_name',
       'brof_code', 'brof_name'],
      dtype='object')

In [314]:
joined_df['created_at'] = 'now()'
joined_df['updated_at'] = 'now()'

In [315]:
desired_order = [
    'unit_name',
    'unit_code',
    'std_date',
    'traffic_amount',
    'inout_name',
    'center_code',
    'center_name',
    'tcs_car_type_cd',
    'brof_code',
    'brof_name',
    'tcs_car_type_nm',
    'tcs_car_type_grp_nm',
    'tcs_car_type_grp_cd',
    'created_at',
    'updated_at'
]

joined_df = joined_df[desired_order]
joined_df[(joined_df['center_name'] == '전북본부') & (joined_df['tcs_car_type_cd'] == '1')]

,unit_name,unit_code,std_date,traffic_amount,inout_name,center_code,center_name,tcs_car_type_cd,brof_code,brof_name,tcs_car_type_nm,tcs_car_type_grp_nm,tcs_car_type_grp_cd,created_at,updated_at
0,완주,544,2025-02-06 15:00:00,245,출구,903.1,전북본부,1,810400,논산,1종,소형차,1,now(),now()
1,소양,545,2025-02-06 15:00:00,12,입구,903.1,전북본부,1,810700,진안,1종,소형차,1,now(),now()
2,소양,545,2025-02-06 15:00:00,127,입구,903.1,전북본부,1,810700,진안,1종,소형차,1,now(),now()
3,소양,545,2025-02-06 15:00:00,16,출구,903.1,전북본부,1,810700,진안,1종,소형차,1,now(),now()
4,소양,545,2025-02-06 15:00:00,151,출구,903.1,전북본부,1,810700,진안,1종,소형차,1,now(),now()
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118628,동전주,590,2025-02-25 10:00:00,217,입구,903.1,전북본부,1,810700,진안,1종,소형차,1,now(),now()
1118629,동전주,590,2025-02-25 10:00:00,14,출구,903.1,전북본부,1,810700,진안,1종,소형차,1,now(),now()
1118630,동전주,590,2025-02-25 10:00:00,145,출구,903.1,전북본부,1,810700,진안,1종,소형차,1,now(),now()
1118661,양촌,599,2025-02-25 10:00:00,75,입구,903.1,전북본부,1,810400,논산,1종,소형차,1,now(),now()


## Insert to table

In [321]:
with engine.begin() as conn:
    insert_query = sa.text('''
    INSERT INTO public.traffic_data_by_unit (
        unit_name,
        unit_code,
        std_date,
        traffic_amount,
        inout_name,
        center_code,
        center_name,
        tcs_car_type_cd,
        brof_code,
        brof_name,
        tcs_car_type_nm,
        tcs_car_type_grp_nm,
        tcs_car_type_grp_cd,
        created_at,
        updated_at
    )
    VALUES (
        :unit_name,
        :unit_code,
        :std_date,
        :traffic_amount,
        :inout_name,
        :center_code,
        :center_name,
        :tcs_car_type_cd,
        :brof_code,
        :brof_name,
        :tcs_car_type_nm,
        :tcs_car_type_grp_nm,
        :tcs_car_type_grp_cd,
        :created_at,
        :updated_at
    )
    ON CONFLICT (unit_code, std_date, inout_name, tcs_car_type_cd)
    DO UPDATE SET 
        unit_name=EXCLUDED.unit_name, 
        traffic_amount=EXCLUDED.traffic_amount, 
        center_code=EXCLUDED.center_code, 
        center_name=EXCLUDED.center_name, 
        brof_code=EXCLUDED.brof_code, 
        brof_name=EXCLUDED.brof_name, 
        tcs_car_type_nm=EXCLUDED.tcs_car_type_nm, 
        tcs_car_type_grp_nm=EXCLUDED.tcs_car_type_grp_nm, 
        tcs_car_type_grp_cd=EXCLUDED.tcs_car_type_grp_cd, 
        created_at=EXCLUDED.created_at, 
        updated_at=EXCLUDED.updated_at
    ''')
    conn.execute(insert_query, joined_df.to_dict(orient='records'))